In [1]:
BASE_PATH = ".."
RAW_PATH = f"{BASE_PATH}/data/raw"
CLEAN_PATH = f"{BASE_PATH}/data/clean"


In [2]:
import pandas as pd
import numpy as np

orders = pd.read_csv(f"{RAW_PATH}/olist_orders_dataset.csv")

In [13]:
orders.head()


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


In [14]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   order_id                       99441 non-null  object
 1   customer_id                    99441 non-null  object
 2   order_status                   99441 non-null  object
 3   order_purchase_timestamp       99441 non-null  object
 4   order_approved_at              99281 non-null  object
 5   order_delivered_carrier_date   97658 non-null  object
 6   order_delivered_customer_date  96476 non-null  object
 7   order_estimated_delivery_date  99441 non-null  object
dtypes: object(8)
memory usage: 6.1+ MB


In [15]:
orders.describe(include='all')

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
count,99441,99441,99441,99441,99281,97658,96476,99441
unique,99441,99441,8,98875,90733,81018,95664,459
top,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2018-08-02 12:05:26,2018-02-27 04:31:10,2018-05-09 15:48:00,2018-05-08 19:36:48,2017-12-20 00:00:00
freq,1,1,96478,3,9,47,3,522


In [18]:
orders.isnull().sum()


order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                 160
order_delivered_carrier_date     1783
order_delivered_customer_date    2965
order_estimated_delivery_date       0
dtype: int64

Observed missing timestamps in approval and delivery stages, likely representing cancelled or in-progress orders. These records were retained to preserve funnel and logistics analysis integrity.

In [27]:
orders.duplicated().sum()

np.int64(0)

In [32]:
orders["order_id"].duplicated().sum()


np.int64(0)

No duplicate order records were found at both row-level and primary key level.

As observed all attributes are of object datatype. Now we'll Validate and enforce appropriate data types across identifiers, categorical variables, and timestamps to ensure accuracy in downstream temporal and aggregation analysis.

In [36]:
# IDs
orders["order_id"] = orders["order_id"].astype("string")
orders["customer_id"] = orders["customer_id"].astype("string")

# Status as category
orders["order_status"] = orders["order_status"].astype("category")

# Date columns
date_cols = [
    "order_purchase_timestamp",
    "order_approved_at",
    "order_delivered_carrier_date",
    "order_delivered_customer_date",
    "order_estimated_delivery_date"
]

for col in date_cols:
    orders[col] = pd.to_datetime(orders[col], errors="coerce")

orders.dtypes


order_id                         string[python]
customer_id                      string[python]
order_status                           category
order_purchase_timestamp         datetime64[ns]
order_approved_at                datetime64[ns]
order_delivered_carrier_date     datetime64[ns]
order_delivered_customer_date    datetime64[ns]
order_estimated_delivery_date    datetime64[ns]
dtype: object

In [37]:
orders["order_status"].value_counts()


order_status
delivered      96478
shipped         1107
canceled         625
unavailable      609
invoiced         314
processing       301
created            5
approved           2
Name: count, dtype: int64

## Ensuring Timestamp Validity

In [38]:
invalid_dates = orders[
    (orders["order_delivered_customer_date"] < orders["order_purchase_timestamp"])
]

invalid_dates.shape


(0, 8)

In [39]:
orders["is_delivered"] = orders["order_status"] == "delivered"


In [44]:
orders.shape


(99441, 9)

In [45]:
orders.isnull().sum()

order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                 160
order_delivered_carrier_date     1783
order_delivered_customer_date    2965
order_estimated_delivery_date       0
is_delivered                        0
dtype: int64

In [46]:
orders.dtypes

order_id                         string[python]
customer_id                      string[python]
order_status                           category
order_purchase_timestamp         datetime64[ns]
order_approved_at                datetime64[ns]
order_delivered_carrier_date     datetime64[ns]
order_delivered_customer_date    datetime64[ns]
order_estimated_delivery_date    datetime64[ns]
is_delivered                               bool
dtype: object

In [47]:
import os

os.makedirs("../data/processed", exist_ok=True)

orders.to_csv(
    "../data/processed/orders_cleaned.csv",
    index=False
)


## Customers Data

In [3]:
customers = pd.read_csv(f"{RAW_PATH}/olist_customers_dataset.csv")

In [4]:
customers.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


In [5]:
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   customer_id               99441 non-null  object
 1   customer_unique_id        99441 non-null  object
 2   customer_zip_code_prefix  99441 non-null  int64 
 3   customer_city             99441 non-null  object
 4   customer_state            99441 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.8+ MB


In [6]:
customers.isnull().sum()


customer_id                 0
customer_unique_id          0
customer_zip_code_prefix    0
customer_city               0
customer_state              0
dtype: int64

No Missing values

In [7]:
customers.duplicated().sum()


np.int64(0)

No duplicate data

## Validating Data Type

In [9]:
customers["customer_id"] = customers["customer_id"].astype("string")
customers["customer_unique_id"] = customers["customer_unique_id"].astype("string")
customers["customer_zip_code_prefix"] = customers["customer_zip_code_prefix"].astype("string")
customers["customer_city"] = customers["customer_city"].astype("string")
customers["customer_state"] = customers["customer_state"].astype("category")
customers.dtypes


customer_id                 string[python]
customer_unique_id          string[python]
customer_zip_code_prefix    string[python]
customer_city               string[python]
customer_state                    category
dtype: object

In [12]:
customers["customer_city"] = (
    customers["customer_city"]
    .str.strip()
    .str.lower()
)



In [13]:
customers["customer_state"].value_counts()


customer_state
SP    41746
RJ    12852
MG    11635
RS     5466
PR     5045
SC     3637
BA     3380
DF     2140
ES     2033
GO     2020
PE     1652
CE     1336
PA      975
MT      907
MA      747
MS      715
PB      536
PI      495
RN      485
AL      413
SE      350
TO      280
RO      253
AM      148
AC       81
AP       68
RR       46
Name: count, dtype: int64

Validated customer-level data by checking identifier uniqueness, standardizing location fields, and enforcing appropriate data types while preserving repeat-customer relationships

In [17]:
customers.duplicated().sum()
customers["customer_id"].duplicated().sum()


np.int64(0)

In [18]:
customers.shape

(99441, 5)

In [20]:
import os
os.makedirs("../data/processed", exist_ok=True)

customers.to_csv(
    "../data/processed/customers_cleaned.csv",
    index=False
)

Cleaned customer master data by validating identifier integrity, enforcing appropriate data types, and standardizing location fields. No missing or duplicate records were observed.

## Products Data

In [21]:
products = pd.read_csv(f"{RAW_PATH}/olist_products_dataset.csv")

In [22]:
products.head()

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0


In [23]:
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32951 entries, 0 to 32950
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   product_id                  32951 non-null  object 
 1   product_category_name       32341 non-null  object 
 2   product_name_lenght         32341 non-null  float64
 3   product_description_lenght  32341 non-null  float64
 4   product_photos_qty          32341 non-null  float64
 5   product_weight_g            32949 non-null  float64
 6   product_length_cm           32949 non-null  float64
 7   product_height_cm           32949 non-null  float64
 8   product_width_cm            32949 non-null  float64
dtypes: float64(7), object(2)
memory usage: 2.3+ MB


In [24]:
products.isnull().sum()

product_id                      0
product_category_name         610
product_name_lenght           610
product_description_lenght    610
product_photos_qty            610
product_weight_g                2
product_length_cm               2
product_height_cm               2
product_width_cm                2
dtype: int64

In [25]:
products["product_category_name"] = (
    products["product_category_name"]
    .fillna("unknown")
    .str.strip()
    .str.lower()
)


In [28]:
products.columns = (
    products.columns
    .str.strip()
    .str.lower()
)


In [38]:
numeric_cols = [
    "product_name_lenght",
    "product_description_lenght",
    "product_photos_qty",
    "product_weight_g",
    "product_length_cm",
    "product_height_cm",
    "product_width_cm"
]

for col in numeric_cols:
    products[col] = pd.to_numeric(products[col], errors="coerce")

In [39]:
products[numeric_cols].isnull().sum()


product_name_lenght           610
product_description_lenght    610
product_photos_qty            610
product_weight_g                2
product_length_cm               2
product_height_cm               2
product_width_cm                2
dtype: int64

Observed minimal missing values in product physical dimensions. These records were retained without imputation to avoid introducing bias, and will be conditionally filtered during logistics-specific analysis.

In [40]:
products.duplicated().sum()
products["product_id"].duplicated().sum()

np.int64(0)

In [41]:
products["product_id"] = products["product_id"].astype("string")
products["product_category_name"] = products["product_category_name"].astype("category")
products.dtypes

product_id                    string[python]
product_category_name               category
product_name_lenght                  float64
product_description_lenght           float64
product_photos_qty                   float64
product_weight_g                     float64
product_length_cm                    float64
product_height_cm                    float64
product_width_cm                     float64
dtype: object

In [42]:
import os
os.makedirs("../data/processed", exist_ok=True)

products.to_csv(
    "../data/processed/products_cleaned.csv",
    index=False
)


Cleaned product-level data by standardizing category labels, validating numeric attributes, and enforcing appropriate data types while retaining incomplete records for analysis-specific filtering.

In [43]:
payments = pd.read_csv("../data/raw/olist_order_payments_dataset.csv")



In [44]:
payments.head()

,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71
3,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78
4,42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.45


In [45]:
payments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103886 entries, 0 to 103885
Data columns (total 5 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   order_id              103886 non-null  object 
 1   payment_sequential    103886 non-null  int64  
 2   payment_type          103886 non-null  object 
 3   payment_installments  103886 non-null  int64  
 4   payment_value         103886 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 4.0+ MB
